In [105]:
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
from pypfopt import expected_returns
from pypfopt import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
from sqlalchemy import create_engine
import numpy as np
import sklearn as sk

In [113]:
engine = create_engine('sqlite:///GDS_IndividualProject.db', echo=False)

df_sample = pd.read_sql_table('sample', con=engine, index_col='Date')
df_mkt_cap = pd.read_sql_table('market_cap', con=engine, index_col='Date')
df_sector = pd.read_sql_table('sectors', con=engine)
df_emissions = pd.read_sql_table('emissions', con=engine)
df_esg = pd.read_sql_table('esg', con=engine)

In [114]:
for column in df_sample.columns:
    df_sample[[column]] = df_sample[[column]].apply(
        lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True, random_state=123), x))

In [185]:
df_benchmark_weights = (df_mkt_cap/(df_mkt_cap.sum(axis=1).values.reshape(-1,1))).tail(1).values
benchmark_returns = pd.DataFrame((df_benchmark_weights*df_sample).sum(axis=1), columns=['Benchmark Returns'])

In [184]:
mean_historical_return(benchmark_returns, returns_data=True, frequency=12)

Benchmark Returns    0.165324
dtype: float64

In [186]:
mu = mean_historical_return(df_sample, frequency=12, returns_data=True)
S = sk.covariance.LedoitWolf(assume_centered=False).fit(df_sample).covariance_

In [187]:
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
weights = ef.max_sharpe()

In [188]:
df_weights = pd.DataFrame(weights,index=['Weight']).T.sort_values(by='Weight', ascending=False)
df_weights[df_weights['Weight']>0]

,Weight
CHK.O,0.271608
OTIS.K,0.219058
BAH,0.105264
PGR,0.103398
CTVA.K,0.089192
PWR,0.061907
RLI,0.036290
ABT,0.036190
WK,0.033117
ETSY.O,0.024484


In [ ]:
carbon_intensity = df_emissions.loc[df_emissions['RIC'].isin(random_RIC)].sort_values(by='RIC')['Carbon Intensity']
(df_benchmark_weights*carbon_intensity.values).sum(axis=1)

Date
2017-12    73.030524
2018-01    72.163586
2018-02    70.151479
2018-03    69.973479
2018-04    69.948099
             ...    
2022-08    44.470038
2022-09    44.031863
2022-10    44.423537
2022-11    45.690087
2022-12    45.275235
Length: 61, dtype: float64

In [ ]:
ef_constrained1 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained1.add_objective(objective_functions.ex_post_tracking_error)
ef_constrained1.add_constraint(lambda x : x >= 0.01)